# Model setup

here we set the model up and explain how it works

imports and load data

In [1]:
# all of this is explained in notebook 0
%run imports.py
from modelinter.preprocessing.imports_load_data import read_csvs, extract_arrays
raw_data = read_csvs()
arrays = extract_arrays(raw_data)

/home/orazio/.virtualenvs/model_interconnectedness_36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


We'll simulate a portfolio of N stocks and N options. We'll model the price of stocks, and feed the output of this model to the model for options.

## Stocks model

The model for **stocks** is a linear regression of their returns against S&P500:

$r_{it} = \alpha_i + \beta_i I_t + \epsilon_{it} $.

with:

* $r_{it}$ - returns of stock $i$ at time $t$
* $I_{t}$ - returns of the index $I$ (S&P500) at time $t$
* $\alpha_i$ - zero order coefficient for stock $i$
* $\beta_i$ - first-order coefficient for stock $i$
* $\epsilon_{it}$ - residuals of the regression for stock $i$ at time $t$

We expect $\alpha_{i}$ to be negligible for stocks, therefore we should be able to approximate:

$r_{it} \approx \beta_i I_t $

We'll call this the **A** model. A more accurate prediction would entail sampling from the distribution of these random variables:

$r_{it} \approx \beta_i I_t + \epsilon_{it} $

where we assume that the $\epsilon$ terms are normally distributed, centered on zero $E[\epsilon_{it}] = 0$, and uncorrelated ($Cov[\epsilon_{it}, \epsilon_{jt}] = 0$ $\forall i \neq j $). We will call this the **B** model, and we'll evaluate it by estimating the standard deviation of the residuals of the regression, and sampling from a normal distribution with adequate parametrisation.

Finally, an even better model, which we'll call the **C** model, would involve removing the assumption of uncorrelated residuals: $\Sigma_{ij} \equiv Cov[\epsilon_{it}, \epsilon_{jt}] \neq 0$. We will estimate this model by evaluating the covariance matrix $\Sigma_{ij}$ with the GLASSO algorithm.


___

#### extrapolating forward in time

this model, being evaluated on daily returns, will predict daily returns. We want to understand what happens on timescales that are characteristic of the CCAR scenario (~4 years). In order to do so, we will have to chain daily predictions. Let's suppose that we want to predict the returns of stock at time $T = D \Delta t$. We can write the returns on index $I$ at time $T$ as the sum of the returns on all days:

$I_T = \prod_t^D (1 + I_t) \approx \sum_t^D I_t$

Mind that this formula works approximately for linear returns, while it's exact for log returns. Similarly for the stock:

$r_T = \prod_t^D (1 + r_t) \approx \sum_t^D r_t \\
= \sum_t^D (\beta_i I_{t} + \epsilon_{it}) = \beta_i I_T + \sum_t^D \epsilon_{it}$

This is important, because the rules for calculating uncertainty follow from it:

$\sigma(r_T) = \sigma(\sum_t^D \epsilon_{it}) = \sqrt{D} \sigma(\epsilon_i)$

where we made the hypothesis that $\epsilon_{it}$  is a stationary process, i.e. $\epsilon_{it} = \epsilon_{i}$, and that it follows a normal distribution.


## Options model

For the **options**, we'll consider one put option per stock, with moneyness ($S/K$) of 1.1, that ends in five years. We'll assume a risk free rate of return of 0.25%. The model will be the original black-scholes.

$C(S_t, t) = N(d_1)S_t - N(d_2) Ke^{-r(T - t)} \\
     d_1 = \frac{1}{\sigma\sqrt{T - t}}\left[\ln\left(\frac{S_t}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)(T - t)\right] \\
     d_2 = d_1 - \sigma\sqrt{T - t}$

with:
* $N(\cdot)$ is the cumulative distribution function of the standard normal distribution
* $\tau \equiv T - t$ is the time to maturity (expressed in years), with $T$ the maturity date, and $t$ current time
* $S$ is the spot price of the underlying asset
* $K$ is the strike price
* $r$ is the risk free rate (annual rate, expressed in terms of continuous compounding)
* $\sigma$ is the volatility of returns of the underlying asset

and all values are intended to be calculated at time $t$.

___

We will model the volatility $\sigma$ with a linear regression on the VIX index similar to the one for the stocks:

$\sigma_{it} = \eta_i + \theta_i J_t + \delta_{it} $

therefore we should have an **A** model like this:

$\sigma_{it} \approx \eta_i + \theta_i J_t $,

where $J_t$ is the VIX index. We will also define, in analogy with the stocks, an **B** model:

$\sigma_{it} \approx \eta_i + \theta_i J_t + \delta_{it}$,

where we consider the $\delta$ to be normally distributed with mean 0, and standard deviation measured from the residuals of the regression. Finally, we'll define a **C** model where $\Sigma_{ij} \equiv Cov[\delta_{it}, \delta_{jt}] \neq 0$.

---

We won't use implied volatility $VI_t$ data directly to estimate the $\theta_i$ and parametrize the model. We will instead assume that realized future volatility $\sigma_{t+p}$ as a proxy for it, i.e. we assume:

$VI_t \propto \sigma_{t+p}$.

For this reason, we equivalently delayed  VIX by $p=-10$ days, following the result of preliminary analysis.

___

#### extrapolating forward in time

The model for implied volatilities doesn't predict returns, but volatilities directly. The value for implied volatility at time $T$ cannot be expressed as a composition of changes along time, therefore the uncertainty for the predictions doesn't increase over time:

$std(\sigma_T) = std(\delta_i)$

and of course we made the implicit hypothesis the hypothesis that $\delta_{it}$  is a stationary process, i.e. $\delta_{it} = \delta_{i}$, and that it follows a normal distribution.

let's define the functions that evaluate the model.

## Code review
All the code is in the modelinter.pgm module. let's go through it.

In [2]:
from modelinter.models.pgm import \
    BasePgmParams, StocksPgmParams, \
    StocksPgm, StocksPgmA, \
    StocksPgmB, StocksPgmC, OptionsPgm

the parameters of the PGM will be stored in an object of class `BasePgmParams`, that only has the logic for saving/loading the parameters and estimating GLASSO on the residuals

In [3]:
view_code(BasePgmParams)

In file: ../../modelinter/models/pgm.py


class BasePgmParams():
    """base model for both stocks and options. It encodes
    save/load functionality, and the logic for calculating lasso."""

    def save(self, name=''):
        Pkl.save(self, Paths.SAVE_DIR.value + self.modelname + name + Paths.PKL_EXT.value)

    def load(self, name=''):
        selfload = Pkl.load(Paths.SAVE_DIR.value + self.modelname + name + Paths.PKL_EXT.value)
        for attr in [_ for _ in selfload.__dir__() if not _.startswith('__')]:
            setattr(self, attr, getattr(selfload, attr))

    def estimate_glasso(self, alpha='cv', **kwargs):
        if 'mode' not in kwargs:
            kwargs['mode'] = 'cd'
        # estimate covariance matrix with GLASSO
        self.alpha = alpha
        if alpha == 'cv':
            self.glasso = GraphLassoCV(**kwargs)
        else:
            self.glasso = GraphLasso(alpha=alpha, **kwargs)
        self.glasso.fit(self.residuals)
        self.Sigma_ij = self.glasso.cov

The realization of a PGM parameters object for the stocks model is a `StocksPgmParams` object.  On initialization it estimates the parameters for the model from the regression, and then calculates GLASSO on the residuals.

In [4]:
view_code(StocksPgmParams)

In file: ../../modelinter/models/pgm.py


class StocksPgmParams(BasePgmParams):
    """This class evaluates and contains the parameters for the stocks PGM."""
    modelname = '_stock_PGM_'

    def __init__(self, dow_t, r_it, alpha='cv', glasso_args=dict()):
        """Evaluate all the parameters needed for stocks."""
        self.beta_i, drop_alpha, self.epsilon_i, drop_r2 = \
            regression_loop(dow_t, r_it)
        # cap the betas at 3? (more than that and you get
        # stocks that go to negative prices on a big shock)
        # self.beta_i[self.beta_i>3] = 3
        # calculate std of epsilon_i
        self.sigma_epsilon_i = np.array([np.std(_) for _ in self.epsilon_i])
        # save residuals in a numpy array for GLASSO
        self.epsilon_i = np.vstack(self.epsilon_i).T
        self.residuals = self.epsilon_i
        # estimate GLASSO with CV
        self.estimate_glasso(alpha=alpha, **glasso_args)
        self.Sigma_ij = self.glasso.covariance_



When predicting, we'll use a realization of the `StocksPgm` base class. This base class just contains the logic for initialization (it gets passed a `StocksPgmParams` when doing `__init__()`), and the part of the prediction logic that will be called by all subclasses.

In [5]:
view_code(StocksPgm)

In file: ../../modelinter/models/pgm.py


class StocksPgm:
    """This class takes as an input a StocksPgmParams object
    and is the base for all the subclasses that make predictions."""

    def __init__(self, stocks_PGM_params):
        self.params = stocks_PGM_params

    # feed the index value and get the stocks returns
    # note that the model is tuned on daily returns.
    # if you want the variance for retuns over more than a day
    # you need to scale the variance!
    # returns are invariant instead
    def predict(self, dow_t):
        return dow_t * self.params.beta_i



The actual realizations of `StocksPgm` are the three kinds of model A, B and C discussed both above and in the paper: `StocksPgmA`, `StocksPgmB` and `StocksPgmC`.

`StocksPgmA` just takes the prediction method from the base class, and changes the signature a little:

In [6]:
view_code(StocksPgmA)

In file: ../../modelinter/models/pgm.py


class StocksPgmA(StocksPgm):
    """This class makes predictions for stocks in PGM A"""

    # days, samples are needed in the signature to standardize interface
    def predict(self, dow_t, days=1, samples=1):
        return super().predict(dow_t)



`StocksPgmB` and `StocksPgmC`'s `predict()` methods instead take the base prediction and generate the residuals.

In [7]:
view_code(StocksPgmB); view_code(StocksPgmC)

In file: ../../modelinter/models/pgm.py


class StocksPgmB(StocksPgm):
    """This class makes predictions for stocks in PGM B"""

    def predict(self, dow_t, days=1, samples=1):
        res = super().predict(dow_t) \
              + np.random.normal(scale=np.sqrt(days) * self.params.sigma_epsilon_i,
                                 size=(samples, self.params.sigma_epsilon_i.size))
        # flatten output if only one sample was requested.
        if samples == 1:
            return res.flatten()
        else:
            return res

In file: ../../modelinter/models/pgm.py


class StocksPgmC(StocksPgm):
    """This class makes predictions for stocks in PGM C"""

    def predict(self, dow_t, days=1, samples=1):
        res = super().predict(dow_t) \
              + np.random.multivariate_normal(
            mean=np.zeros(self.params.Sigma_ij.shape[0]),
            cov=days * self.params.Sigma_ij,
            size=samples)
        # flatten output if only one sample was requested.
     

The options model is structured the same way, with

* `BasePgmParams`
    * `OptionsPgmParams`
* `OptionsPgm`
    * `optionsPgmA`
    * `optionsPgmB`
    * `optionsPgmC`
    
the logic in `OptionsPgm` is a bit more convoluted, though, because its `predict()` method contains the code to calculate options prices through the Black-Scholes formula, which is structured as two ugly nested loops. The logic for generating volatilities is instead found in the subclasses.

In [9]:
view_code(OptionsPgm)

In file: ../../modelinter/models/pgm.py


class OptionsPgm():
    """This class takes as an input a options_PGM_params object
    and is the base for all the subclasses that make predictions."""

    def __init__(self, options_PGM_params):
        self.params = options_PGM_params

    # feed the index value and get the option prices.
    def predict(self, vix, r, strike_price, stocks_prices, tau_i, flag_i, moneyness_i, samples=1):
        """provides the  base logic for calculating a PGM prediction.
        The only thing that changes between calculation of PGMS A,B and C
        is the way you predict sigma_i, and we put that logic into
        subclasses."""
        # eval daily volatility and ANNUALIZE
        sigma_i = self.sigma_i(vix=vix, samples=samples) * Const.ANNUALIZE.value
        # then predict all the options values:
        options_prices = []
        # optimization: If one wants multiple samples from models B or C
        # it's much faster (~100 times) to draw all of t

## Test calculation
Now let's test that they actually work:

In [15]:
#DefaultSettings is just a namespace containing some settings for eval_PGM
from modelinter.models.calculations import eval_PGM, DefaultSettings
#we'll also need a couple of constants
from modelinter.models.constants import Const
import numpy as np

In [11]:
#val_pgm is instead a function that estimates all the model
#parameters from the data:
view_code(eval_PGM)

In file: ../../modelinter/models/calculations.py


def eval_PGM(arrays, DefaultSettings):
    """evalutes parameters for all PGM models from data"""
    PGM_stock_params = StocksPgmParams(arrays.sp_r,
                                       arrays.stocks_r,
                                       alpha=DefaultSettings.alpha)
    PGM_stock_A = StocksPgmA(PGM_stock_params)
    PGM_stock_B = StocksPgmB(PGM_stock_params)
    PGM_stock_C = StocksPgmC(PGM_stock_params)
    PGM_options_params = options_PGM_params(arrays.vix,
                                            arrays.sigma,
                                            alpha=DefaultSettings.alpha)
    PGM_options_A = OptionsPgmA(PGM_options_params)
    PGM_options_B = OptionsPgmB(PGM_options_params)
    PGM_options_C = OptionsPgmC(PGM_options_params)
    return SimpleNamespace(
        PGM_stock_params=PGM_stock_params,
        PGM_stock_A=PGM_stock_A,
        PGM_stock_B=PGM_stock_B,
        PGM_stock_C=PGM_stock_C,
        PGM_options_p

In [12]:
#let's evaluate them
models = eval_PGM(arrays, DefaultSettings)

/home/orazio/.virtualenvs/model_interconnectedness_36/lib/python3.6/site-packages/sklearn/covariance/graph_lasso_.py:230: RuntimeWarning: invalid value encountered in multiply
  * coefs)
/home/orazio/.virtualenvs/model_interconnectedness_36/lib/python3.6/site-packages/sklearn/covariance/graph_lasso_.py:232: RuntimeWarning: invalid value encountered in multiply
  * coefs)
/home/orazio/.virtualenvs/model_interconnectedness_36/lib/python3.6/site-packages/numpy/linalg/linalg.py:1741: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
/home/orazio/.virtualenvs/model_interconnectedness_36/lib/python3.6/site-packages/numpy/linalg/linalg.py:1741: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)


In [13]:
#does it work?
#prediction for stocks
(models.PGM_stock_A.predict(.01)[:20],
 models.PGM_stock_B.predict(.01)[:20],
 models.PGM_stock_C.predict(.01)[:20])

(array([ 0.0134131 ,  0.01664561,  0.00855781,  0.0100695 ,  0.00787624,
         0.01116747,  0.01093633,  0.01341751,  0.01261105,  0.01307771,
         0.00984361,  0.0122559 ,  0.01881287,  0.00368643,  0.00399366,
         0.01289216,  0.01084731,  0.00843793,  0.01002991,  0.01248531]),
 array([ 0.01303737,  0.04084867,  0.00164628,  0.02008065, -0.00414546,
         0.01698921,  0.01993527,  0.01517178,  0.01017546,  0.01117527,
         0.01872408,  0.00727516, -0.02223713,  0.00331236, -0.006212  ,
        -0.0130239 ,  0.00181805,  0.01186242,  0.03578859,  0.03516007]),
 array([-0.00575096, -0.01140658, -0.041516  ,  0.02313669,  0.00550467,
        -0.00213905, -0.00589475,  0.01292658,  0.00673286,  0.03681896,
        -0.00927839, -0.01595387,  0.01520671,  0.00345395, -0.00961339,
         0.02931361,  0.01351047,  0.00896937, -0.00119406,  0.02666732]))

In [19]:
#let's set an arbitrary level for vix
vix_set = np.mean(arrays.vix)
#prediction for options
(
models.PGM_options_A.predict(
                vix=vix_set,
                r=Const.RISK_FREE_RATE.value,
                strike_price = None,
                stocks_prices = arrays.stocks_p[-1],
                tau_i = [1]*arrays.stocks_p.shape[-1],
                flag_i = ['p']*arrays.stocks_p.shape[-1],
                moneyness_i=1.1,
                )[:20],
models.PGM_options_B.predict(
                vix=vix_set,
                r=Const.RISK_FREE_RATE.value,
                strike_price = None,
                stocks_prices = arrays.stocks_p[-1],
                tau_i = [1]*arrays.stocks_p.shape[-1],
                flag_i = ['p']*arrays.stocks_p.shape[-1],
                moneyness_i=1.1,
                )[:20],
models.PGM_options_C.predict(
                vix=vix_set,
                r=Const.RISK_FREE_RATE.value,
                strike_price = None,
                stocks_prices = arrays.stocks_p[-1],
                tau_i = [1]*arrays.stocks_p.shape[-1],
                flag_i = ['p']*arrays.stocks_p.shape[-1],
                moneyness_i=1.1,
                )[:20]
)

(array([  2.25637023,   4.21749353,   9.15615605,   5.25452619,
          4.56891578,   1.84272017,   4.57236283,   5.06469782,
          3.39962771,   2.96651475,   2.71164893,  19.9510826 ,
          6.1874537 ,   1.63071977,   1.86491825,   0.77411904,
          6.21560687,   1.71684084,  16.77283157,   2.64817515]),
 array([  2.50048853,   3.16116217,  13.42410234,   3.88597932,
          0.13487924,   1.73301509,   3.34999359,   5.11234817,
          1.94589061,   2.64342236,   1.96265213,   7.37813502,
          6.09119396,   1.99908815,   3.3721076 ,   0.72012092,
          4.27581201,   3.16350274,   2.84534272,   2.62853147]),
 array([  1.66912642,   6.44886533,   9.75660033,   4.27468483,
          3.09478039,   1.80269938,   3.46930963,   5.03584118,
          2.76676655,   4.16531723,   2.43365515,  24.31817609,
         13.26333298,   0.89872765,   1.68125526,   0.96172057,
          6.65184947,   1.44445721,  21.82793259,   4.83455257]))